In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
from utils import tile_raster_images
import math
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['image.cmap'] = 'jet'

# Task 1

Implement the RBM that uses CD-1 for training. For input data use MNIST numbers. The visible layer must then have 784 elements, and the hidden layer should have 100 elements. Since the values of the input samples (image) are real numbers in the range [0 1], they can be used as $p(v_i=1)$, so for the initial values of the visible layer, sampling should be performed. Set the mini batch size to 100 samples, and the number of epochs to 100.

Subtasks:

1. Visualize the weights of $W$ obtained by training and try to interpret the weights associated with some hidden neurons.
2. Visualize the reconstruction results of the first 20 MNIST samples. Visualize the values of $p(v_i=1)=σ(∑^N_{j=1} w_{ji}h_j+a_i)$ instead of the binary values obtained by sampling.
3. Examine the activation frequency of hidden layer elements and visualize the learned weights of $W$ sorted by the frequency
4. Skip the initial sampling/binarization based on the real input data, and use the original input data (real numbers from the range [0 1]) as input layer $v$. How different is such RBM from the previous one?
5. Increase the number of Gibs sampling in CDs. What are the differences?
6. Examine the effects of varying the learning constant.
7. Randomly initialize the hidden layer, run a few Gibbs samplings, and visualize the generated visible layer
8. Perform above experiments with a smaller and a larger number of hidden neurons. What do you observe about weights and reconstructions?

Use the following template with the utility file [utils.py](https://dlunizg.github.io/assets/lab4/utils.py).

REMARK: In addition to filling out the missing code, the template should be tailored as needed, and can be customized freely. So please be especially careful with the claims that some of the code is not working for you!

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trainX, trainY, testX, testY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

In [ ]:
def init_weights(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def init_bias(shape):
    initial = tf.zeros(shape, dtype=tf.float32)
    return tf.Variable(initial)


def sample_prob(probs):
    """Sample vector x by probability vector p (x = 1) = probs"""
    return tf.to_float(tf.random_uniform(tf.shape(probs)) <= probs)


def draw_weights(W, shape, N, stat_shape, interpolation="bilinear"):
    """Visualization of weight
     W - weight vector
     shape - tuple dimensions for 2D weight display - usually input image dimensions, eg (28,28)
     N - number weight vectors
     shape_state - Dimension for 2D state display (eg for 100 states (10,10)
    """
    image = (tile_raster_images(
        X=W.T,
        img_shape=shape,
        tile_shape=(int(math.ceil(N/stat_shape[0])), stat_shape[0]),
        tile_spacing=(1, 1)))
    plt.figure(figsize=(10, 14))
    plt.imshow(image, interpolation=interpolation)
    plt.axis('off')
    
    
def draw_reconstructions(ins, outs, states, shape_in, shape_state, N):
    """Visualization of inputs and associated reconstructions and hidden layer states
     ins -- input vectors
     outs - reconstructed vectors
     states - hidden layer state vectors
     shape_in - dimension of input images eg (28,28)
     shape_state - Dimension for 2D state display (eg for 100 states (10,10)
     N - number of samples
    """
    plt.figure(figsize=(8, int(2 * N)))
    for i in range(N):
        plt.subplot(N, 4, 4*i + 1)
        plt.imshow(ins[i].reshape(shape_in), vmin=0, vmax=1, interpolation="nearest")
        plt.title("Test input")
        plt.axis('off')
        plt.subplot(N, 4, 4*i + 2)
        plt.imshow(outs[i][0:784].reshape(shape_in), vmin=0, vmax=1, interpolation="nearest")
        plt.title("Reconstruction")
        plt.axis('off')
        plt.subplot(N, 4, 4*i + 3)
        plt.imshow(states[i].reshape(shape_state), vmin=0, vmax=1, interpolation="nearest")
        plt.title("States")
        plt.axis('off')
    plt.tight_layout()

    
def draw_generated(stin, stout, gen, shape_gen, shape_state, N):
    """Visualization of initial hidden states, final hidden states and associated reconstructions
     stin - the initial hidden layer
     stout - reconstructed vectors
     gen - vector of hidden layer state
     shape_gen - dimensional input image eg (28,28)
     shape_state - Dimension for 2D state display (eg for 100 states (10,10)
     N - number of samples
    """
    plt.figure(figsize=(8, int(2 * N)))
    for i in range(N):

        plt.subplot(N, 4, 4*i + 1)
        plt.imshow(stin[i].reshape(shape_state), vmin=0, vmax=1, interpolation="nearest")
        plt.title("set state")
        plt.axis('off')
        plt.subplot(N, 4, 4*i + 2)
        plt.imshow(stout[i][0:784].reshape(shape_state), vmin=0, vmax=1, interpolation="nearest")
        plt.title("final state")
        plt.axis('off')
        plt.subplot(N, 4, 4*i + 3)
        plt.imshow(gen[i].reshape(shape_gen), vmin=0, vmax=1, interpolation="nearest")
        plt.title("generated visible")
        plt.axis('off')
    plt.tight_layout()

    
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def draw_rec(inp, title, size, Nrows, in_a_row, j):
    """ Draw an iteration of creating the visible layer
     inp - visible layer
     title - thumbnail title
     size - 2D dimensions of visible layer
     Nrows - max. number of thumbnail rows 
     in-a-row. number of thumbnails in one row
     j - position of thumbnails in the grid
    """
    plt.subplot(Nrows, in_a_row, j)
    plt.imshow(inp.reshape(size), vmin=0, vmax=1, interpolation="nearest")
    plt.title(title)
    plt.axis('off')
    
    
def reconstruct(ind, v_shape, h_shape, states, orig, weights, biases):
    """ Sequential visualization of  the visible layer reconstruction
     ind - index of digits in orig (matrix with digits as lines)
     states - state vectors of input vectors
     orig - original input vectors
     weights - weight matrix
    """
    j = 1
    in_a_row = 6
    Nimg = states.shape[1] + 3
    Nrows = int(np.ceil(float(Nimg+2)/in_a_row))
    
    plt.figure(figsize=(12, 2*Nrows))
       
    draw_rec(states[ind], 'states', h_shape, Nrows, in_a_row, j)
    j += 1
    draw_rec(orig[ind], 'input', v_shape, Nrows, in_a_row, j)
    
    reconstr = biases.copy()
    j += 1
    draw_rec(sigmoid(reconstr), 'biases', v_shape, Nrows, in_a_row, j)
    
    for i in range(Nh):
        if states[ind,i] > 0:
            j += 1
            reconstr = reconstr + weights[:,i]
            titl = '+= s' + str(i+1)
            draw_rec(sigmoid(reconstr), titl, v_shape, Nrows, in_a_row, j)
    plt.tight_layout()
    

In [ ]:
def get_w_grad(v, h):    
    v = tf.reshape(v, [-1, int(v.shape[1]), 1])
    h = tf.reshape(h, [-1, 1, int(h.shape[1])])
    return tf.reduce_sum(tf.matmul(v, h), reduction_indices=0) 

In [ ]:

class Task1():
    '''
    Nh - number of elements of the hidden layer (e.g. 100)
    h_shape - shape of hidden layer (e.g. 10x10)
    Nv - number of elements of the visible layer (e.g. 784)
    v_shape - shape of visible layer (e.g. 28x28)
    Nu - number of samples for visualization of reconstruction
    gibbs_sampling_steps - number of iterations of Gibbs sampling
    alpha - multiplier for weight update
    rnd_hidden_init - set hidden unit randomly, not based on visible layer
    '''
    def __init__(self, Nh, h_shape, Nv, v_shape, Nu, gibbs_sampling_steps, alpha, rnd_hidden_init=False):
        self.g1 = tf.Graph()
        with self.g1.as_default():
            
            self.v = tf.placeholder("float", [None, Nv]) # visible layer
            self.w = init_weights([Nv, Nh]) # weights
            self.a = init_bias([Nv]) # bias for visible layer
            self.b = init_bias([Nh]) # bias for hidden layer

            #https://towardsdatascience.com/deep-learning-meets-physics-restricted-boltzmann-machines-part-i-6df5c4918c15
            # p(h) = sigma(b + vw)
            self.h_prob = tf.sigmoid(tf.matmul(self.v, self.w) + tf.transpose(self.b))
            if rnd_hidden_init:
                self.h = tf.to_float(tf.random_uniform(tf.shape(self.h_prob)))
            else:
                self.h = sample_prob(self.h_prob)
                
            # Gibbs sampling
            self.h_next = self.h
            for step in range(gibbs_sampling_steps):
                self.gibbs_sample_one()

                #Implement the RBM that uses CD-1 for training.
                # http://deeplearning.net/tutorial/rbm.html
                self.constrastive_divergence()

                
            self.v_next_prob = tf.sigmoid(tf.matmul(self.h_next, tf.transpose(self.w)) + tf.transpose(self.a))
            self.v_next = sample_prob(self.v_next_prob)

            err1 = self.v - self.v_next_prob
            self.err_sum1 = tf.reduce_mean(err1 * err1)

            initialize1 = tf.global_variables_initializer()
    
    
            self.sess1 = tf.Session(graph=self.g1)
            self.sess1.run(initialize1)
        
        
    def gibbs_sample_one(self):
        '''
        Gibbs sampling
        -> we take an input vector v_0 and use it to predict the values of the hidden state h_0.
        The hidden state are used on the other hand to predict new input state v. This procedure
        is repeated k times. This procedure is illustrated in Fig. 2.
        @ https://towardsdatascience.com/deep-learning-meets-physics-restricted-boltzmann-machines-part-ii-4b159dce1ffb
        '''
        self.v_next_prob = tf.sigmoid(tf.matmul(self.h_next, tf.transpose(self.w)) + tf.transpose(self.a))
        self.v_next = sample_prob(self.v_next_prob)
        self.h_next_prob = tf.sigmoid(tf.matmul(self.v_next, self.w) + tf.transpose(self.b))
        self.h_next = sample_prob(self.h_next_prob)
        
    
    def constrastive_divergence(self):
        '''
        Contrastive Divergence @ towardsdatascience
        The update of the weight matrix happens during the Contrastive Divergence step.
        Vectors v_0 and v_k are used to calculate the activation probabilities for
        hidden values h_0 and h_k (Eq.4).

        Using the update matrix the new weights can be calculated with gradient ascent, given by:
        delta_w = w_positive_grad - w_negative_grad
        w_new = w_old + delta_w*alpha
        a_new = a_old + delta_a*alpha
        b_new = b_old + delta_b*alpha
        '''
        w_positive_grad = get_w_grad(self.v, self.h_prob)
        w_negative_grad = get_w_grad(self.v_next_prob, self.h_next_prob)

        delta_w = (w_positive_grad - w_negative_grad) / tf.to_float(tf.shape(self.v)[0]) # .shape[0] - number of rows

        update_w = tf.assign_add(self.w, alpha * delta_w)
        update_a = tf.assign_add(self.a, alpha * tf.reduce_mean(self.v - self.v_next, 0))
        update_b = tf.assign_add(self.b, alpha * tf.reduce_mean(self.h - self.h_next, 0)) 
        
        self.out1 = (update_w, update_a, update_b)
        
        
    def train(self, total_batch):
        print('training, total batch:', total_batch)
        for i in range(total_batch):
            batch, label = mnist.train.next_batch(batch_size)
            err, _ = self.sess1.run([self.err_sum1, self.out1], feed_dict={self.v: batch})

            if i%(int(total_batch/10)) == 0:
                print('iter={:<8} error={:2.8f}'.format(i, err))

        print('iter={:<8} error={:2.8f}'.format(i, err))
        #self.w_s = self.w.eval(session=self.sess1)
        #self.a_s = self.a.eval(session=self.sess1)
        #self.b_s = self.b.eval(session=self.sess1)
        self.w_s, self.a_s, self.b_s = self.sess1.run([self.w, self.a, self.b])
        self.vr, self.h_next_s = self.sess1.run([self.v_next_prob, self.h_next], feed_dict={self.v: testX[0:Nu,:]})
    
    
    def get_training_variables(self):
        return self.w_s, self.h_next_s, self.vr, self.a_s, self.b_s

    
    def run_h_next(self, _input):
        out_1 = self.sess1.run((self.v_next), feed_dict={self.h: _input})
        return out_1

    
    def run_v_next(self, _input):
        out_1_prob, out_1, hout1 = self.sess1.run((self.v_next_prob, self.v_next, self.h_next), feed_dict={self.v: _input})
        return out_1_prob, out_1, hout1

    

In [ ]:
# initialization of class - RBM network

batch_size = 100
epochs = 100
n_samples = mnist.train.num_examples
total_batch = int(n_samples / batch_size) * epochs


Nh = 100 # The number of elements of the hidden layer
h_shape = (10,10)
Nv = 784 # The number of elements of the visible layer
v_shape = (28,28)
Nu = 5000 # Number of samples for visualization of reconstruction

gibbs_sampling_steps = 1
alpha = 0.1

rbm = Task1(Nh, h_shape, Nv, v_shape, Nu, gibbs_sampling_steps, alpha)

In [ ]:
print('total batch:', total_batch)
total_batch = 100#int(total_batch/55)
print('total batch:', total_batch)

rbm.train(total_batch)


In [ ]:
print('Subtask 1')
print('visualization of weights')
# visualization of weights
w_states, h_next_states, v_reconstructions, a_states, b_states = rbm.get_training_variables()
draw_weights(w_states, v_shape, Nh, h_shape)


In [ ]:
print('Subtask 2')
print('visualization of reconstructions of 20 MNIST samples')
# visualization of reconstructions and states
draw_reconstructions(testX, v_reconstructions, h_next_states, v_shape, h_shape, 20)


In [ ]:
# visualization of a reconstructions with the gradual addition of the contributions of active hidden elements
reconstruct(0, v_shape, h_shape, h_next_states, testX, w_states, a_states) # the first argument is the digit index in the digit matrix


In [ ]:
# The probability that the hidden state is included through Nu input samples
'''
plt.figure()
tmp = (h1s.sum(0)/h1s.shape[0]).reshape(h1_shape)
plt.imshow(tmp, vmin=0, vmax=1, interpolation="nearest")
plt.axis('off')
plt.colorbar()
plt.title('likelihood of the activation of certain neurons of the hidden layer')
'''


In [ ]:
print('Subtask 3')
print('Visualization of weights sorted by frequency')
# Visualization of weights sorted by frequency
tmp = (h_next_states.sum(0)/h_next_states.shape[0]).reshape(h_shape)
tmp_ind = (-tmp).argsort(None)
draw_weights(w_states[:, tmp_ind], v_shape, Nh, h_shape)
plt.title('Sorted weight matrices - from most to the least used')


In [ ]:
print('Subtask 4')
print('Generating samples from random vectors')
# Generating samples from random vectors
r_input = np.random.rand(100, Nh)
r_input[r_input > 0.9] = 1 # percentage of active - vary freely
r_input[r_input < 1] = 0
r_input = r_input * 20 # Boosting in case a small percentage is active

s = 10
for i in range(10):
    r_input[i,:] = 0
    r_input[i,i]= s

out_1 = rbm.run_h_next(r_input)
#out_1_prob, out_1, hout1 = sess1.run((v1_prob, v1, h1), feed_dict={h0: r_input})
#out_1 = sess1.run((v1), feed_dict={h0: r_input})


In [ ]:
#draw_generated(r_input, hout1, out_1_prob, v_shape, h1_shape, 10)


In [ ]:
print('Subtask 5')
print('Emulation of additional Gibbs sampling using feed_dict')
# Emulation of additional Gibbs sampling using feed_dict
for i in range(1000):
    out_1_prob, out_1, hout1 = rbm.run_v_next(out_1)
    if i%100 == 0:
        print(i)

draw_generated(r_input, hout1, out_1_prob, v_shape, h_shape, 10)

In [ ]:
print('Subtask 5')
print('Increase the number of Gibs sampling')

total_batch = 100
alpha = 0.1

gibbs_sampling_steps = 1
print('\ngibbs_sampling_steps {}'.format(gibbs_sampling_steps))
rbm = Task1(Nh, h_shape, Nv, v_shape, Nu, gibbs_sampling_steps, alpha)
rbm.train(total_batch)

gibbs_sampling_steps = 2
print('\ngibbs_sampling_steps {}'.format(gibbs_sampling_steps))
rbm = Task1(Nh, h_shape, Nv, v_shape, Nu, gibbs_sampling_steps, alpha)
rbm.train(total_batch)

gibbs_sampling_steps = 3
print('\ngibbs_sampling_steps {}'.format(gibbs_sampling_steps))
rbm = Task1(Nh, h_shape, Nv, v_shape, Nu, gibbs_sampling_steps, alpha)
rbm.train(total_batch)

gibbs_sampling_steps = 4
print('\ngibbs_sampling_steps {}'.format(gibbs_sampling_steps))
rbm = Task1(Nh, h_shape, Nv, v_shape, Nu, gibbs_sampling_steps, alpha)
rbm.train(total_batch)

In [ ]:
print('Subtask 6')
print('different learning steps')

gibbs_sampling_steps = 1
total_batch = 100

alpha = 0.01
print('\nalpha {}'.format(alpha))
rbm2 = Task1(Nh, h_shape, Nv, v_shape, Nu, gibbs_sampling_steps, alpha)
rbm2.train(total_batch)

alpha = 0.1
print('\nalpha {}'.format(alpha))
rbm2 = Task1(Nh, h_shape, Nv, v_shape, Nu, gibbs_sampling_steps, alpha)
rbm2.train(total_batch)

alpha = 0.2
print('\nalpha {}'.format(alpha))
rbm2 = Task1(Nh, h_shape, Nv, v_shape, Nu, gibbs_sampling_steps, alpha)
rbm2.train(total_batch)

alpha = 0.4
print('\nalpha {}'.format(alpha))
rbm2 = Task1(Nh, h_shape, Nv, v_shape, Nu, gibbs_sampling_steps, alpha)
rbm.train(total_batch)


In [ ]:
print('Subtask 7')
print('Randomly initialize the hidden layer, run a few Gibbs samplings, and visualize the generated visible layer')


batch_size = 100
epochs = 100
n_samples = mnist.train.num_examples
total_batch = int(n_samples / batch_size) * epochs

Nh = 100 # The number of elements of the first hidden layer
h_shape = (10,10)
Nv = 784 # The number of elements of the first visible layer
v_shape = (28,28)
Nu = 5000 # Number of samples for visualization of reconstruction

gibbs_sampling_steps = 1
alpha = 0.1

total_batch = 100 #int(total_batch/55)

print('\nrandom hidden initialization=False')
rbm = Task1(Nh, h_shape, Nv, v_shape, Nu, gibbs_sampling_steps, alpha, rnd_hidden_init=True)
rbm.train(total_batch)
w_states, h_next_states, v_reconstructions, a_states, b_states = rbm.get_training_variables()
draw_weights(w_states, v_shape, Nh, h_shape)
plt.title('random hidden initialization=False')

print('\nrandom hidden initialization=True')
rbm = Task1(Nh, h_shape, Nv, v_shape, Nu, gibbs_sampling_steps, alpha, rnd_hidden_init=False)
rbm.train(total_batch)
w_states, h_next_states, v_reconstructions, a_states, b_states = rbm.get_training_variables()
draw_weights(w_states, v_shape, Nh, h_shape)
plt.title('random hidden initialization=True')

In [ ]:
print('Subtask 8')
print('Perform above experiments with a smaller and a larger number of hidden neurons.')


batch_size = 100
epochs = 100
n_samples = mnist.train.num_examples
total_batch = int(n_samples / batch_size) * epochs


print('25 (5x5) neurons in hidden layer')

Nh = 25 # The number of elements of the hidden layer
h_shape = (5, 5)
Nv = 784 # The number of elements of the visible layer
v_shape = (28,28)
Nu = 5000 # Number of samples for visualization of reconstruction

gibbs_sampling_steps = 1
alpha = 0.1

total_batch = 100 #int(total_batch/55)

rbm = Task1(Nh, h_shape, Nv, v_shape, Nu, gibbs_sampling_steps, alpha)
rbm.train(total_batch)
w_states, h_next_states, v_reconstructions, a_states, b_states = rbm.get_training_variables()
draw_reconstructions(testX, v_reconstructions, h_next_states, v_shape, h_shape, 20)


In [ ]:

print('400 (20x20) neurons in hidden layer')

Nh = 400 # The number of elements of the hidden layer
h_shape = (20, 20)
Nv = 784 # The number of elements of the visible layer
v_shape = (28,28)
Nu = 5000 # Number of samples for visualization of reconstruction

gibbs_sampling_steps = 1
alpha = 0.1

total_batch = 100 #int(total_batch/55)

rbm = Task1(Nh, h_shape, Nv, v_shape, Nu, gibbs_sampling_steps, alpha)
rbm.train(total_batch)
w_states, h_next_states, v_reconstructions, a_states, b_states = rbm.get_training_variables()
draw_reconstructions(testX, v_reconstructions, h_next_states, v_shape, h_shape, 20)

# Task 2

Deep Belief Network (DBN) is a deep network that is obtained by stacking multiple RBMs one to another, each of which is trained greedily with inputs from the hidden (“outgoing”) layer of the previous RBM (except the first RBM being trained directly with input samples). In theory, such DBN should increase $p(v)$ which is our initial goal. The use of DBN, ie reconstruction of the input sample, is carried out according to the scheme below. In the upward pass, hidden layers are determined from the visible layer until the highest RBM is reached, then the CD algorithm is executed, then in the downward direction, the lower hidden layers are determined until the visible layer. The weights between the individual layers are the same in the upward and in the downward pass. Implement a three-layer DBN that consists of two greedy RBMs. The first RBM should be as in 1st task, and the second RBM should have a hidden layer of 100 elements.

Subtasks:

1. Visualize the weights of $W1$ and $W2$ obtained by training.
2. Visualize the results of the reconstruction of the first 20 MNIST samples.
3. Randomly initialize the topmost hidden layer, run a few Gibbs samplings, and visualize generated visible layer patterns - compare with the previous task.
4. Set the number of hidden layer elements of the upper RBM to the number of lower RBM visible layer elements, and set the initial weights $W_2$ to $W_1^T$. What are the effects of change? Visualize elements of the topmost layer as 28x28 matrix.

In [ ]:

class Task2():
    '''
    Nh - number of elements of the hidden layer (e.g. 100)
    h_shape - shape of hidden layer (e.g. 10x10)
    Nv - number of elements of the visible layer (e.g. 784)
    v_shape - shape of visible layer (e.g. 28x28)
    Nu - number of samples for visualization of reconstruction
    gibbs_sampling_steps - number of iterations of Gibbs sampling
    alpha - multiplier for weight update
    w_prev - weights from previous RBN
    a_prev - bias for visible layer from previous RBN
    b_prev - bias for hidden layer from previous RBN
    rnd_hidden_init - set hidden unit randomly, not based on visible layer
    '''
    def __init__(self, Nh, h_shape, Nv, v_shape, Nu, gibbs_sampling_steps, alpha, w_prev, a_prev, b_prev, rnd_hidden_init=False):
        self.g2 = tf.Graph()
        with self.g2.as_default():
            
            self.v = tf.placeholder("float", [None, Nv]) # visible layer
            self.w = init_weights([Nv, Nh])
            self.a = init_bias([Nv])
            self.b = init_bias([Nh])
            self.w_prev = tf.Variable(w_prev)
            self.a_prev = tf.Variable(a_prev)
            self.b_prev = tf.Variable(b_prev)

            #https://towardsdatascience.com/deep-learning-meets-physics-restricted-boltzmann-machines-part-i-6df5c4918c15
            # p(h) = sigma(b + vw)
            self.h_prob = tf.sigmoid(tf.matmul(self.v, self.w) + tf.transpose(self.b))
            if rnd_hidden_init:
                self.h = tf.to_float(tf.random_uniform(tf.shape(self.h_prob)))
            else:
                self.h = sample_prob(self.h_prob)
                
            # Gibbs sampling
            self.h_next = self.h
            for step in range(gibbs_sampling_steps):
                self.gibbs_sample_one()

                #Implement the RBM that uses CD-1 for training.
                # http://deeplearning.net/tutorial/rbm.html
                self.constrastive_divergence()

                
            self.v_next_prob = tf.sigmoid(tf.matmul(self.h_next, tf.transpose(self.w)) + tf.transpose(self.a))
            self.v_next = sample_prob(self.v_next_prob)
            
            err2 = self.v - self.v_next_prob
            self.err_sum2 = tf.reduce_mean(err2 * err2)

            initialize2 = tf.global_variables_initializer()
    
    
            self.sess2 = tf.Session(graph=self.g2)
            self.sess2.run(initialize2)
        
        
    def gibbs_sample_one(self):
        '''
        Gibbs sampling
        -> we take an input vector v_0 and use it to predict the values of the hidden state h_0.
        The hidden state are used on the other hand to predict new input state v. This procedure
        is repeated k times. This procedure is illustrated in Fig. 2.
        @ https://towardsdatascience.com/deep-learning-meets-physics-restricted-boltzmann-machines-part-ii-4b159dce1ffb
        '''
        self.v_next_prob = tf.sigmoid(tf.matmul(self.h_next, tf.transpose(self.w)) + tf.transpose(self.a))
        self.v_next = sample_prob(self.v_next_prob)
        self.h_next_prob = tf.sigmoid(tf.matmul(self.v_next, self.w) + tf.transpose(self.b))
        self.h_next = sample_prob(self.h_next_prob)
        
    
    def constrastive_divergence(self):
        '''
        Contrastive Divergence @ towardsdatascience
        The update of the weight matrix happens during the Contrastive Divergence step.
        Vectors v_0 and v_k are used to calculate the activation probabilities for
        hidden values h_0 and h_k (Eq.4).

        Using the update matrix the new weights can be calculated with gradient ascent, given by:
        delta_w = w_positive_grad - w_negative_grad
        w_new = w_old + delta_w*alpha
        a_new = a_old + delta_a*alpha
        b_new = b_old + delta_b*alpha
        '''
        w_positive_grad = get_w_grad(self.v, self.h_prob)
        w_negative_grad = get_w_grad(self.v_next_prob, self.h_next_prob)

        delta_w = (w_positive_grad - w_negative_grad) / tf.to_float(tf.shape(self.v)[0]) # .shape[0] - number of rows

        update_w = tf.assign_add(self.w, alpha * delta_w)
        update_a = tf.assign_add(self.a, alpha * tf.reduce_mean(self.v - self.v_next, 0))
        update_b = tf.assign_add(self.b, alpha * tf.reduce_mean(self.h - self.h_next, 0)) 
        
        self.out2 = (update_w, update_a, update_b)
        
        
    def train(self, total_batch):
        print('training, total batch:', total_batch)
        for i in range(total_batch):
            batch, label = mnist.train.next_batch(batch_size)
            err, _ = self.sess2.run([self.err_sum2, self.out2], feed_dict={self.v: batch})

            if i%(int(total_batch/10)) == 0:
                print('iter={:<8} error={:2.8f}'.format(i, err))

        print('iter={:<8} error={:2.8f}'.format(i, err))
        #self.w_s = self.w.eval(session=self.sess2)
        #self.a_s = self.a.eval(session=self.sess2)
        #self.b_s = self.b.eval(session=self.sess2)
        self.w_s, self.a_s, self.b_s = self.sess2.run([self.w, self.a, self.b])
        self.vr, self.h_next_s = self.sess2.run([self.v_next_prob, self.h_next], feed_dict={self.v: testX[0:Nu,:]})
    
    
    def get_training_variables(self):
        return self.w_s, self.h_next_s, self.vr, self.a_s, self.b_s

    
    def run_h_next(self, _input):
        out_2 = self.sess2.run((self.v_next), feed_dict={self.h: _input})
        return out_2

    
    def run_v_next(self, _input):
        out_2_prob, out_2, hout2 = self.sess2.run((self.v_next_prob, self.v_next, self.h_next), feed_dict={self.v: _input})
        return out_2_prob, out_2, hout2

    

In [ ]:
# initialization of class - RBM network

batch_size = 100

Nh = 100 # The number of elements of the hidden layer
h_shape = (10,10)
Nv = 784 # The number of elements of the visible layer
v_shape = (28,28)
Nu = 5000 # Number of samples for visualization of reconstruction

gibbs_sampling_steps = 1
alpha = 0.1

total_batch = 100


print('First layer of RBM')
rbm1 = Task1(Nh, h_shape, Nv, v_shape, Nu, gibbs_sampling_steps, alpha)
rbm1.train(total_batch)
w_states, h_next_states, v_reconstructions, a_states, b_states = rbm1.get_training_variables()


In [ ]:
print('Second layer of RBM')
rbm2 = Task2(Nh, h_shape, Nv, v_shape, Nu, gibbs_sampling_steps, alpha, w_states, a_states, b_states)
rbm2.train(total_batch)
w_states, h_next_states, v_reconstructions, a_states, b_states = rbm2.get_training_variables()

In [ ]:
print('Subtask 1')
print('Visualize the weights of W1 and W2 obtained by training')

w_states, h_next_states, v_reconstructions, a_states, b_states = rbm1.get_training_variables()
draw_weights(w_states, v_shape, Nh, h_shape)
plt.title('Weights - first RBM')

w_states, h_next_states, v_reconstructions, a_states, b_states = rbm2.get_training_variables()
draw_weights(w_states, v_shape, Nh, h_shape)
plt.title('Weights - second RBM')

In [ ]:
print('Subtask 2')
print('Visualize the results of the reconstruction of the first 20 MNIST samples')

w_states, h_next_states, v_reconstructions, a_states, b_states = rbm1.get_training_variables()
draw_reconstructions(testX, v_reconstructions, h_next_states, v_shape, h_shape, 20)
plt.title('Reconstructions after first RBM')


In [ ]:

w_states, h_next_states, v_reconstructions, a_states, b_states = rbm2.get_training_variables()
draw_reconstructions(testX, v_reconstructions, h_next_states, v_shape, h_shape, 20)
plt.title('Reconstructions after second RBM')

In [ ]:
print('Subtask 3')
print('Randomly initialize the topmost hidden layer')

total_batch = 100

rbm2 = Task2(Nh, h_shape, Nv, v_shape, Nu, gibbs_sampling_steps, alpha, w_states, a_states, b_states, rnd_hidden_init=True)
rbm2.train(total_batch)
w_states, h_next_states, v_reconstructions, a_states, b_states = rbm2.get_training_variables()
draw_weights(w_states, v_shape, Nh, h_shape)
plt.title('Weights - second RBM with random hidden initialization')

In [ ]:
print('Subtask 4')
print('Set the number of hidden layer elements of the upper RBM to the number of lower RBM visible layer elements,')

total_batch = 100
Nh = 784#100 # The number of elements of the hidden layer
h_shape = (28,28)#(10,10)

rbm2 = Task2(Nh, h_shape, Nv, v_shape, Nu, gibbs_sampling_steps, alpha, w_states, a_states, b_states, rnd_hidden_init=True)
rbm2.train(total_batch)
w_states, h_next_states, v_reconstructions, a_states, b_states = rbm2.get_training_variables()
draw_weights(w_states, v_shape, Nh, h_shape)
plt.title('Weights - second RBM with hidden size = visible size of first RBM')